Expected Inside 50 Model

In [1]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-i50-model/")

In [2]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_i50_model.config import modelling_file_path
from expected_i50_model.src.modelling_data_contract import ModellingDataContract
from expected_i50_model.src.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_i50_model.src.modelling.supermodel import SuperXGBClassifier
from expected_i50_model.src.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Model Version

In [34]:
model_version = 1
model_name = 'expected_i50'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-i50-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-i50-model/data/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-i50-model/model_outputs/preprocessors/"

In [4]:
RESPONSE = ModellingDataContract.RESPONSE

In [5]:
FEATURES = ModellingDataContract.feature_list

In [6]:
MONOTONE_CONSTRAINTS = ModellingDataContract.monotone_constraints

Load Data

In [7]:
df_modelling = pd.read_csv(modelling_file_path)
df_modelling.tail()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,quarter_a0,quarter_seconds_a0,overall_seconds_a0,start_x_a0,start_y_a0,end_x_a0,end_y_a0,dx_a0,dy_a0,movement_a0,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,start_x_a1,start_y_a1,end_x_a1,end_y_a1,dx_a1,dy_a1,movement_a1,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,start_x_a2,start_y_a2,end_x_a2,end_y_a2,dx_a2,dy_a2,movement_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02,Inside50,Inside50TrainingSet,Inside50TestSet,Inside50ValidationSet
54732,2023F4_Collingwood_BrisbaneLions,226,1752.0,4,1461.0,7449.0,Collingwood,Josh Daicos,14.0,-16.0,-48.0,16.0,Kick Inside 50,clanger,4,1461.0,7449.0,14.0,-16.0,-48.0,16.0,-62.0,32.0,69.771054,True,False,False,4,1461.0,7449.0,14.0,-16.0,14.0,-16.0,0.0,0.0,0.000000,True,False,False,4,1460.0,7448.0,13.0,-11.0,14.0,-16.0,1.0,-5.0,5.099020,True,True,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False,True,False
54733,2023F4_Collingwood_BrisbaneLions,229,1771.0,4,1491.0,7479.0,Collingwood,Steele Sidebottom,11.0,15.0,71.0,21.0,Kick Inside 50,effective,4,1491.0,7479.0,11.0,15.0,71.0,21.0,60.0,6.0,60.299254,True,False,False,4,1490.0,7478.0,10.0,15.0,11.0,15.0,1.0,0.0,1.000000,False,False,True,4,1489.0,7477.0,-24.0,-0.0,11.0,15.0,35.0,15.0,38.078866,True,False,-1.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,1,True,False,False
54734,2023F4_Collingwood_BrisbaneLions,231,1784.0,4,1554.0,7542.0,Collingwood,Steele Sidebottom,25.0,63.0,-69.0,-31.0,Kick Inside 50,effective,4,1554.0,7542.0,25.0,63.0,-69.0,-31.0,-94.0,-94.0,132.936075,True,False,False,4,1543.0,7531.0,30.0,60.0,25.0,63.0,-5.0,3.0,5.830952,True,False,False,4,1542.0,7530.0,15.0,42.0,25.0,63.0,10.0,21.0,23.259407,True,True,-11.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0,1,True,False,False
54735,2023F4_Collingwood_BrisbaneLions,233,1799.0,4,1610.0,7598.0,Brisbane Lions,Lachie Neale,-10.0,-9.0,-45.0,-0.0,Kick Inside 50,effective,4,1610.0,7598.0,-10.0,-9.0,-45.0,-0.0,-35.0,9.0,36.138622,True,False,False,4,1605.0,7593.0,-8.0,-17.0,-10.0,-9.0,-2.0,8.0,8.246211,False,True,False,4,1595.0,7583.0,-20.0,-14.0,-10.0,-9.0,10.0,5.0,11.180340,True,True,-5.0,-15.0,0.0,0.0,0.0,0.0,0.0,0.0,1,False,True,False
54736,2023F4_Collingwood_BrisbaneLions,236,1822.0,4,1707.0,7695.0,Brisbane Lions,Zac Bailey,29.0,31.0,-47.0,-10.0,Kick Inside 50,ineffective,4,1707.0,7695.0,29.0,31.0,-47.0,-10.0,-76.0,-41.0,86.353923,True,False,False,4,1706.0,7694.0,32.0,28.0,29.0,31.0,-3.0,3.0,4.242641,True,False,False,4,1705.0,7693.0,28.0,21.0,29.0,31.0,1.0,10.0,10.049876,True,True,-1.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False,False,True


In [8]:
training_data = df_modelling[df_modelling[RESPONSE+"TrainingSet"]]
test_data = df_modelling[df_modelling[RESPONSE+"TestSet"]]
cal_data = df_modelling[df_modelling[RESPONSE+"ValidationSet"]]

In [9]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]
X_cal, y_cal = cal_data.drop(columns=[RESPONSE]), cal_data[RESPONSE]

In [10]:
X_test.head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,quarter_a0,quarter_seconds_a0,overall_seconds_a0,start_x_a0,start_y_a0,end_x_a0,end_y_a0,dx_a0,dy_a0,movement_a0,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,start_x_a1,start_y_a1,end_x_a1,end_y_a1,dx_a1,dy_a1,movement_a1,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,start_x_a2,start_y_a2,end_x_a2,end_y_a2,dx_a2,dy_a2,movement_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02,Inside50TrainingSet,Inside50TestSet,Inside50ValidationSet
1,202101_BrisbaneLions_Sydney,2,23.0,1,108.0,108.0,Brisbane Lions,Zac Bailey,37.0,56.0,46.0,22.0,Kick Inside 50,ineffective,1,108.0,108.0,37.0,56.0,46.0,22.0,9.0,-34.0,35.171011,True,False,False,1,108.0,108.0,34.0,55.0,37.0,56.0,3.0,1.0,3.162278,False,True,False,1,104.0,104.0,34.0,41.0,37.0,56.0,3.0,15.0,15.297059,True,True,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,False
2,202101_BrisbaneLions_Sydney,7,72.0,1,234.0,234.0,Brisbane Lions,Mitch Robinson,29.0,-23.0,60.0,-31.0,Kick Inside 50,ineffective,1,234.0,234.0,29.0,-23.0,60.0,-31.0,31.0,-8.0,32.015621,True,False,False,1,232.0,232.0,26.0,-23.0,29.0,-23.0,3.0,0.0,3.000000,True,False,False,1,231.0,231.0,26.0,-10.0,29.0,-23.0,3.0,-13.0,13.341664,True,True,-2.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,False
13,202101_BrisbaneLions_Sydney,29,305.0,1,964.0,964.0,Brisbane Lions,Daniel Rich,45.0,-51.0,50.0,-5.0,Kick Inside 50,ineffective,1,964.0,964.0,45.0,-51.0,50.0,-5.0,5.0,46.0,46.270941,True,False,False,1,960.0,960.0,36.0,-54.0,45.0,-51.0,9.0,3.0,9.486833,True,False,False,1,959.0,959.0,21.0,-46.0,45.0,-51.0,24.0,-5.0,24.515301,True,True,-4.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,False
23,202101_BrisbaneLions_Sydney,55,543.0,1,1670.0,1670.0,Sydney,Dane Rampe,13.0,60.0,54.0,26.0,Kick Inside 50,clanger,1,1670.0,1670.0,13.0,60.0,54.0,26.0,41.0,-34.0,53.263496,True,False,False,1,1665.0,1665.0,24.0,58.0,13.0,60.0,-11.0,2.0,11.180340,True,False,False,1,1659.0,1659.0,-72.0,-16.0,13.0,60.0,85.0,76.0,114.021928,True,False,-5.0,-11.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,False
49,202101_BrisbaneLions_Sydney,124,1095.0,3,222.0,4035.0,Brisbane Lions,Dayne Zorko,7.0,11.0,56.0,4.0,Kick Inside 50,effective,3,222.0,4035.0,7.0,11.0,56.0,4.0,49.0,-7.0,49.497475,True,False,False,3,216.0,4029.0,15.0,9.0,7.0,11.0,-8.0,2.0,8.246211,True,False,False,3,210.0,4023.0,-13.0,-8.0,7.0,11.0,20.0,19.0,27.586228,True,False,-6.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,False


In [11]:
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]
X_cal_preproc = X_cal[FEATURES]

In [12]:
X_train_preproc.shape, X_test_preproc.shape

((35031, 46), (10948, 46))

In [13]:
X_train_preproc.head()

,quarter_a0,quarter_seconds_a0,overall_seconds_a0,start_x_a0,start_y_a0,end_x_a0,end_y_a0,dx_a0,dy_a0,movement_a0,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,start_x_a1,start_y_a1,end_x_a1,end_y_a1,dx_a1,dy_a1,movement_a1,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,start_x_a2,start_y_a2,end_x_a2,end_y_a2,dx_a2,dy_a2,movement_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02
4,1,303.0,303.0,-8.0,44.0,50.0,28.0,58.0,-16.0,60.166436,True,False,False,1,299.0,299.0,-13.0,43.0,-8.0,44.0,5.0,1.0,5.099020,True,False,False,1,297.0,297.0,-18.0,52.0,-8.0,44.0,10.0,-8.0,12.806248,True,True,-4.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,414.0,414.0,29.0,-34.0,70.0,-13.0,41.0,21.0,46.065171,True,False,False,1,407.0,407.0,35.0,-30.0,29.0,-34.0,-6.0,-4.0,7.211103,True,False,False,1,406.0,406.0,13.0,-59.0,29.0,-34.0,16.0,25.0,29.681644,True,True,-7.0,-8.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,548.0,548.0,19.0,-49.0,49.0,-35.0,30.0,14.0,33.105891,True,False,False,1,544.0,544.0,13.0,-50.0,19.0,-49.0,6.0,1.0,6.082763,True,False,False,1,542.0,542.0,21.0,-57.0,19.0,-49.0,-2.0,8.0,8.246211,True,True,-4.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,625.0,625.0,29.0,58.0,62.0,18.0,33.0,-40.0,51.855569,True,False,False,1,620.0,620.0,22.0,57.0,29.0,58.0,7.0,1.0,7.071068,True,False,False,1,620.0,620.0,13.0,56.0,29.0,58.0,16.0,2.0,16.124515,True,True,-5.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,651.0,651.0,5.0,-66.0,43.0,-32.0,38.0,34.0,50.990195,True,False,False,1,651.0,651.0,1.0,-66.0,5.0,-66.0,4.0,0.0,4.000000,True,False,False,1,645.0,645.0,4.0,45.0,5.0,-66.0,1.0,-111.0,111.004504,True,False,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
X_test_preproc.head()

,quarter_a0,quarter_seconds_a0,overall_seconds_a0,start_x_a0,start_y_a0,end_x_a0,end_y_a0,dx_a0,dy_a0,movement_a0,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,start_x_a1,start_y_a1,end_x_a1,end_y_a1,dx_a1,dy_a1,movement_a1,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,start_x_a2,start_y_a2,end_x_a2,end_y_a2,dx_a2,dy_a2,movement_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02
1,1,108.0,108.0,37.0,56.0,46.0,22.0,9.0,-34.0,35.171011,True,False,False,1,108.0,108.0,34.0,55.0,37.0,56.0,3.0,1.0,3.162278,False,True,False,1,104.0,104.0,34.0,41.0,37.0,56.0,3.0,15.0,15.297059,True,True,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,234.0,234.0,29.0,-23.0,60.0,-31.0,31.0,-8.0,32.015621,True,False,False,1,232.0,232.0,26.0,-23.0,29.0,-23.0,3.0,0.0,3.000000,True,False,False,1,231.0,231.0,26.0,-10.0,29.0,-23.0,3.0,-13.0,13.341664,True,True,-2.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0
13,1,964.0,964.0,45.0,-51.0,50.0,-5.0,5.0,46.0,46.270941,True,False,False,1,960.0,960.0,36.0,-54.0,45.0,-51.0,9.0,3.0,9.486833,True,False,False,1,959.0,959.0,21.0,-46.0,45.0,-51.0,24.0,-5.0,24.515301,True,True,-4.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0
23,1,1670.0,1670.0,13.0,60.0,54.0,26.0,41.0,-34.0,53.263496,True,False,False,1,1665.0,1665.0,24.0,58.0,13.0,60.0,-11.0,2.0,11.180340,True,False,False,1,1659.0,1659.0,-72.0,-16.0,13.0,60.0,85.0,76.0,114.021928,True,False,-5.0,-11.0,0.0,0.0,0.0,0.0,0.0,0.0
49,3,222.0,4035.0,7.0,11.0,56.0,4.0,49.0,-7.0,49.497475,True,False,False,3,216.0,4029.0,15.0,9.0,7.0,11.0,-8.0,2.0,8.246211,True,False,False,3,210.0,4023.0,-13.0,-8.0,7.0,11.0,20.0,19.0,27.586228,True,False,-6.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [15]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints=MONOTONE_CONSTRAINTS)

In [16]:
xgb_tuner.training_data.head()

,quarter_a0,quarter_seconds_a0,overall_seconds_a0,start_x_a0,start_y_a0,end_x_a0,end_y_a0,dx_a0,dy_a0,movement_a0,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,start_x_a1,start_y_a1,end_x_a1,end_y_a1,dx_a1,dy_a1,movement_a1,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,start_x_a2,start_y_a2,end_x_a2,end_y_a2,dx_a2,dy_a2,movement_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02
4,1,303.0,303.0,-8.0,44.0,50.0,28.0,58.0,-16.0,60.166436,True,False,False,1,299.0,299.0,-13.0,43.0,-8.0,44.0,5.0,1.0,5.099020,True,False,False,1,297.0,297.0,-18.0,52.0,-8.0,44.0,10.0,-8.0,12.806248,True,True,-4.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,414.0,414.0,29.0,-34.0,70.0,-13.0,41.0,21.0,46.065171,True,False,False,1,407.0,407.0,35.0,-30.0,29.0,-34.0,-6.0,-4.0,7.211103,True,False,False,1,406.0,406.0,13.0,-59.0,29.0,-34.0,16.0,25.0,29.681644,True,True,-7.0,-8.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,548.0,548.0,19.0,-49.0,49.0,-35.0,30.0,14.0,33.105891,True,False,False,1,544.0,544.0,13.0,-50.0,19.0,-49.0,6.0,1.0,6.082763,True,False,False,1,542.0,542.0,21.0,-57.0,19.0,-49.0,-2.0,8.0,8.246211,True,True,-4.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,625.0,625.0,29.0,58.0,62.0,18.0,33.0,-40.0,51.855569,True,False,False,1,620.0,620.0,22.0,57.0,29.0,58.0,7.0,1.0,7.071068,True,False,False,1,620.0,620.0,13.0,56.0,29.0,58.0,16.0,2.0,16.124515,True,True,-5.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,651.0,651.0,5.0,-66.0,43.0,-32.0,38.0,34.0,50.990195,True,False,False,1,651.0,651.0,1.0,-66.0,5.0,-66.0,4.0,0.0,4.000000,True,False,False,1,645.0,645.0,4.0,45.0,5.0,-66.0,1.0,-111.0,111.004504,True,False,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
xgb_tuner.tune_hyperparameters()

[I 2023-10-07 23:11:02,623] A new study created in memory with name: no-name-06e2bf46-7227-4f61-8e5f-838556154ced
[I 2023-10-07 23:11:02,797] Trial 0 finished with value: 0.5511891068293914 and parameters: {'max_depth': 10, 'min_child_weight': 45, 'eta': 0.8935422340772312, 'gamma': 0.011625066657561241, 'lambda': 5.643762804668677, 'alpha': 0.0003326509843453253, 'subsample': 0.7539426719375188, 'colsample_bytree': 0.6474699238042125}. Best is trial 0 with value: 0.5511891068293914.
[I 2023-10-07 23:11:02,949] Trial 1 finished with value: 0.5904705421821819 and parameters: {'max_depth': 6, 'min_child_weight': 75, 'eta': 0.10118003966051048, 'gamma': 1.7074541692976921, 'lambda': 0.00421530399944292, 'alpha': 4.3953392637405635, 'subsample': 0.47024169289729706, 'colsample_bytree': 0.3837896686415827}. Best is trial 0 with value: 0.5511891068293914.
[I 2023-10-07 23:11:03,100] Trial 2 finished with value: 0.5627167460849584 and parameters: {'max_depth': 8, 'min_child_weight': 45, 'eta'

Number of finished trials:  100
Best trial:
  Value: 0.532564073178067
  Params: 
    max_depth: 8
    min_child_weight: 60
    eta: 0.5505481233023182
    gamma: 0.10557260377505404
    lambda: 0.0019409332417870077
    alpha: 0.02413349118162038
    subsample: 0.7075219212823793
    colsample_bytree: 0.812493033979598


In [18]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 8,
 'min_child_weight': 60,
 'eta': 0.5505481233023182,
 'gamma': 0.10557260377505404,
 'lambda': 0.0019409332417870077,
 'alpha': 0.02413349118162038,
 'subsample': 0.7075219212823793,
 'colsample_bytree': 0.812493033979598}

Training Model - SuperXGBClassifier class for training and predictions

In [19]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = MONOTONE_CONSTRAINTS

In [20]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test,
                               X_cal = X_cal_preproc,
                               y_cal = y_cal,
                               params = params)

In [21]:
super_xgb.fit()

[0]	validation_0-logloss:0.58692	validation_1-logloss:0.59346
[1]	validation_0-logloss:0.55335	validation_1-logloss:0.56344
[2]	validation_0-logloss:0.54101	validation_1-logloss:0.55344
[3]	validation_0-logloss:0.53582	validation_1-logloss:0.55036
[4]	validation_0-logloss:0.53026	validation_1-logloss:0.54693
[5]	validation_0-logloss:0.52687	validation_1-logloss:0.54435
[6]	validation_0-logloss:0.52369	validation_1-logloss:0.54308
[7]	validation_0-logloss:0.51999	validation_1-logloss:0.54146
[8]	validation_0-logloss:0.51812	validation_1-logloss:0.54005
[9]	validation_0-logloss:0.51459	validation_1-logloss:0.54020
[10]	validation_0-logloss:0.51296	validation_1-logloss:0.53964
[11]	validation_0-logloss:0.51133	validation_1-logloss:0.53909
[12]	validation_0-logloss:0.50949	validation_1-logloss:0.53917
[13]	validation_0-logloss:0.50720	validation_1-logloss:0.53938
[14]	validation_0-logloss:0.50425	validation_1-logloss:0.54045
[15]	validation_0-logloss:0.50230	validation_1-logloss:0.54060
[1

In [22]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.812493033979598, device=None,
              early_stopping_rounds=50, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.10557260377505404,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.5505481233023182,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=60, missing=nan, monotone_constraints={},
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [23]:
super_xgb.xgb_model.get_booster().feature_names

['quarter_a0',
 'quarter_seconds_a0',
 'overall_seconds_a0',
 'start_x_a0',
 'start_y_a0',
 'end_x_a0',
 'end_y_a0',
 'dx_a0',
 'dy_a0',
 'movement_a0',
 'outcome_effective_a1',
 'outcome_ineffective_a1',
 'outcome_clanger_a1',
 'quarter_a1',
 'quarter_seconds_a1',
 'overall_seconds_a1',
 'start_x_a1',
 'start_y_a1',
 'end_x_a1',
 'end_y_a1',
 'dx_a1',
 'dy_a1',
 'movement_a1',
 'outcome_effective_a2',
 'outcome_ineffective_a2',
 'outcome_clanger_a2',
 'quarter_a2',
 'quarter_seconds_a2',
 'overall_seconds_a2',
 'start_x_a2',
 'start_y_a2',
 'end_x_a2',
 'end_y_a2',
 'dx_a2',
 'dy_a2',
 'movement_a2',
 'team_1',
 'team_2',
 'time_delta1',
 'time_delta2',
 'dx_a01',
 'dy_a01',
 'move_a01',
 'dx_a02',
 'dy_a02',
 'move_a02']

In [24]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [25]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]
cal_probas = super_xgb.predict_proba(X_cal_preproc)[:, 1]

In [26]:
super_xgb.calibrate()

In [27]:
train_cal_probas = super_xgb.predict_proba(X_train_preproc, calibrate=True)
test_cal_probas = super_xgb.predict_proba(X_test_preproc, calibrate=True)

Check Average Predictions

In [28]:
train_probas.mean(), training_data[RESPONSE].mean(), train_cal_probas.mean()

(0.67093563, 0.6716907881590591, 0.668360175558566)

In [29]:
test_probas.mean(), test_data[RESPONSE].mean(), test_cal_probas.mean()

(0.66978765, 0.6690719766167337, 0.6673926178858706)

Export model

In [31]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [35]:
train_info = training_data[['match_id', "chain_number", "order", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export['xgb_probas_cal'] = train_cal_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,match_id,chain_number,order,Inside50,quarter_a0,quarter_seconds_a0,overall_seconds_a0,start_x_a0,start_y_a0,end_x_a0,end_y_a0,dx_a0,dy_a0,movement_a0,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,start_x_a1,start_y_a1,end_x_a1,end_y_a1,dx_a1,dy_a1,movement_a1,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,start_x_a2,start_y_a2,end_x_a2,end_y_a2,dx_a2,dy_a2,movement_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,10,106.0,1,1,303.0,303.0,-8.0,44.0,50.0,28.0,58.0,-16.0,60.166436,True,False,False,1,299.0,299.0,-13.0,43.0,-8.0,44.0,5.0,1.0,5.099020,True,False,False,1,297.0,297.0,-18.0,52.0,-8.0,44.0,10.0,-8.0,12.806248,True,True,-4.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.742396,0.735200
1,202101_BrisbaneLions_Sydney,12,129.0,1,1,414.0,414.0,29.0,-34.0,70.0,-13.0,41.0,21.0,46.065171,True,False,False,1,407.0,407.0,35.0,-30.0,29.0,-34.0,-6.0,-4.0,7.211103,True,False,False,1,406.0,406.0,13.0,-59.0,29.0,-34.0,16.0,25.0,29.681644,True,True,-7.0,-8.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.757487,0.748542
2,202101_BrisbaneLions_Sydney,17,179.0,1,1,548.0,548.0,19.0,-49.0,49.0,-35.0,30.0,14.0,33.105891,True,False,False,1,544.0,544.0,13.0,-50.0,19.0,-49.0,6.0,1.0,6.082763,True,False,False,1,542.0,542.0,21.0,-57.0,19.0,-49.0,-2.0,8.0,8.246211,True,True,-4.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.428338,0.447143
3,202101_BrisbaneLions_Sydney,19,196.0,0,1,625.0,625.0,29.0,58.0,62.0,18.0,33.0,-40.0,51.855569,True,False,False,1,620.0,620.0,22.0,57.0,29.0,58.0,7.0,1.0,7.071068,True,False,False,1,620.0,620.0,13.0,56.0,29.0,58.0,16.0,2.0,16.124515,True,True,-5.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.829126,0.812447
4,202101_BrisbaneLions_Sydney,20,207.0,0,1,651.0,651.0,5.0,-66.0,43.0,-32.0,38.0,34.0,50.990195,True,False,False,1,651.0,651.0,1.0,-66.0,5.0,-66.0,4.0,0.0,4.000000,True,False,False,1,645.0,645.0,4.0,45.0,5.0,-66.0,1.0,-111.0,111.004504,True,False,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.617705,0.624305


In [36]:
test_info = test_data[['match_id', "chain_number", "order", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export['xgb_probas_cal'] = test_cal_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,match_id,chain_number,order,Inside50,quarter_a0,quarter_seconds_a0,overall_seconds_a0,start_x_a0,start_y_a0,end_x_a0,end_y_a0,dx_a0,dy_a0,movement_a0,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,start_x_a1,start_y_a1,end_x_a1,end_y_a1,dx_a1,dy_a1,movement_a1,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,start_x_a2,start_y_a2,end_x_a2,end_y_a2,dx_a2,dy_a2,movement_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,2,23.0,0,1,108.0,108.0,37.0,56.0,46.0,22.0,9.0,-34.0,35.171011,True,False,False,1,108.0,108.0,34.0,55.0,37.0,56.0,3.0,1.0,3.162278,False,True,False,1,104.0,104.0,34.0,41.0,37.0,56.0,3.0,15.0,15.297059,True,True,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.331457,0.350240
1,202101_BrisbaneLions_Sydney,7,72.0,0,1,234.0,234.0,29.0,-23.0,60.0,-31.0,31.0,-8.0,32.015621,True,False,False,1,232.0,232.0,26.0,-23.0,29.0,-23.0,3.0,0.0,3.000000,True,False,False,1,231.0,231.0,26.0,-10.0,29.0,-23.0,3.0,-13.0,13.341664,True,True,-2.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.486327,0.502968
2,202101_BrisbaneLions_Sydney,29,305.0,0,1,964.0,964.0,45.0,-51.0,50.0,-5.0,5.0,46.0,46.270941,True,False,False,1,960.0,960.0,36.0,-54.0,45.0,-51.0,9.0,3.0,9.486833,True,False,False,1,959.0,959.0,21.0,-46.0,45.0,-51.0,24.0,-5.0,24.515301,True,True,-4.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.685237,0.684637
3,202101_BrisbaneLions_Sydney,55,543.0,0,1,1670.0,1670.0,13.0,60.0,54.0,26.0,41.0,-34.0,53.263496,True,False,False,1,1665.0,1665.0,24.0,58.0,13.0,60.0,-11.0,2.0,11.180340,True,False,False,1,1659.0,1659.0,-72.0,-16.0,13.0,60.0,85.0,76.0,114.021928,True,False,-5.0,-11.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.829594,0.812870
4,202101_BrisbaneLions_Sydney,124,1095.0,1,3,222.0,4035.0,7.0,11.0,56.0,4.0,49.0,-7.0,49.497475,True,False,False,3,216.0,4029.0,15.0,9.0,7.0,11.0,-8.0,2.0,8.246211,True,False,False,3,210.0,4023.0,-13.0,-8.0,7.0,11.0,20.0,19.0,27.586228,True,False,-6.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.859062,0.839789


Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")